## Import Required Packages

In [3]:
import numpy as np
from girth.synthetic import create_synthetic_irt_dichotomous
from girth import twopl_mml

## Run the 2PL Model

In [5]:
# Simulation Parameters
n_items = 25
n_people = 500

# Create Synthetic Data
difficulty = np.linspace(-2.5, 2.5, n_items)
discrimination = np.random.rand(n_items) + 0.5
theta = np.random.randn(n_people)

syn_data = create_synthetic_irt_dichotomous(difficulty, discrimination, theta)

# Solve for parameters
results = twopl_mml(syn_data)

## Compare the Results

In [6]:
list(zip(discrimination.round(3), results['Discrimination'].round(3)))

[(0.617, 1.077),
 (0.954, 0.735),
 (0.627, 0.528),
 (1.392, 1.194),
 (1.328, 1.76),
 (1.42, 0.995),
 (1.14, 1.495),
 (0.608, 0.722),
 (1.418, 1.433),
 (1.122, 1.079),
 (1.437, 1.608),
 (1.393, 1.314),
 (0.789, 0.586),
 (0.574, 0.674),
 (0.597, 0.669),
 (1.428, 1.557),
 (0.58, 0.537),
 (1.086, 1.102),
 (1.284, 1.379),
 (1.075, 0.978),
 (1.228, 1.046),
 (1.273, 1.262),
 (0.928, 0.942),
 (1.287, 1.484),
 (0.848, 0.705)]